In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from  keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.applications import mobilenet_v2

In [3]:
base_model = mobilenet_v2.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
Pred = Dense(2,activation='softmax')(x)

model = Model(inputs=base_model.inputs,outputs=Pred)

for layer in model.layers:
    layer.trainable = True

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [4]:
Train_data = image.ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)

Val_data = image.ImageDataGenerator(rescale = 1./255)

In [5]:
Train_gen = Train_data.flow_from_directory('Coviddataset/Train',
                                            target_size=(299,299),
                                            batch_size = 32,
                                           classes = ['Covid','Normal'],
                                            class_mode = 'categorical')

Val_gen = Val_data.flow_from_directory('Coviddataset/Val',
                                        target_size = (299,299),
                                        batch_size = 32,
                                        classes = ['Covid','Normal'],
                                        class_mode = 'categorical')

Found 286 images belonging to 2 classes.
Found 65 images belonging to 2 classes.


In [ ]:
hist = model.fit(Train_gen,
                 steps_per_epoch = 8,
                 epochs = 10,
                 validation_data = Val_gen,
                 validation_steps = 2,
                 
                 )

Epoch 1/10
